In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from collections import Counter
import tensorflow as tf
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
buys = open('C:/Users/Test.PPMUMCPU0034/Desktop/Predictive Analytics/Section 10/yoochoose-buys.dat', 'r')
clicks = open('C:/Users/Test.PPMUMCPU0034/Desktop/Predictive Analytics/Section 10/yoochoose-clicks.dat', 'r')

In [4]:
print("loading data....")
initial_buys_df = pd.read_csv(buys, names=['Session ID', 'Timestamp', 'Item ID', 'Category', 'Quantity'],
                              dtype={'Session ID': 'float32', 'Timestamp': 'str', 'Item ID': 'float32',
                                     'Category': 'str'}) 

initial_clicks_df = pd.read_csv(clicks, names=['Session ID', 'Timestamp', 'Item ID', 'Category'],
                                dtype={'Category': 'str'})
print("Data loaded..!!")

loading data....
Data loaded..!!


In [5]:
initial_buys_df.set_index('Session ID', inplace=True)
initial_clicks_df.set_index('Session ID', inplace=True)

initial_buys_df = initial_buys_df.drop('Timestamp', 1)
initial_clicks_df = initial_clicks_df.drop('Timestamp', 1)

In [6]:
x = Counter(initial_buys_df.index).most_common(100000) 
top_k = dict(x).keys()                                

initial_buys_df = initial_buys_df[initial_buys_df.index.isin(top_k)]  
initial_clicks_df = initial_clicks_df[initial_clicks_df.index.isin(top_k)]

In [7]:
initial_buys_df['_Session ID'] = initial_buys_df.index

In [8]:
transformed_buys = pd.get_dummies(initial_buys_df)
transformed_clicks = pd.get_dummies(initial_clicks_df)

In [9]:
filtered_buys = transformed_buys.filter(regex="Item.*|Category.*")
filtered_clicks = transformed_clicks.filter(regex="Item.*|Category.*")
historical_buy_data = filtered_buys.groupby(filtered_buys.index).sum()
historical_buy_data = historical_buy_data.rename(columns=lambda column_name: 'buy history:' + column_name)
historical_click_data = filtered_clicks.groupby(filtered_clicks.index).sum()
historical_click_data = historical_click_data.rename(columns=lambda column_name: 'click history:' + column_name)